In [11]:
import numpy as np

# Set up parameters for the LoRA matrices
d, k = 100, 100
r_max = 3
np.random.seed(42)

# Create a common basis for all clients
U_base, S_base_full, Vt_base = np.linalg.svd(np.random.rand(d, k))
U_base = U_base[:, :r_max]
Vt_base = Vt_base[:r_max, :]

# Create singular values for each client (r=1, r=2, r=3)
S_A = np.random.rand(1)
S_B = np.random.rand(2)
S_C = np.random.rand(3)

# Define client weights
w_A, w_B, w_C = 0.2, 0.5, 0.3

# --- Aggregation Method 1: Aggregate Singular Values (SVD-based) ---

# Pad singular values with zeros
S_A_padded = np.zeros(r_max)
S_A_padded[:1] = S_A
S_B_padded = np.zeros(r_max)
S_B_padded[:2] = S_B
S_C_padded = S_C

# Compute the aggregated singular values
S_agg_svd = w_A * S_A_padded + w_B * S_B_padded + w_C * S_C_padded

# Reconstruct the final LoRA matrix
final_lora_agg_svd = U_base @ np.diag(S_agg_svd) @ Vt_base

# --- Aggregation Method 2: Aggregate Full Matrices (Parameter-based) ---

# Construct full LoRA matrices for each client from B and A
B_A = U_base[:, :1]
A_A = np.diag(S_A) @ Vt_base[:1, :]

B_B = U_base[:, :2]
A_B = np.diag(S_B) @ Vt_base[:2, :]

B_C = U_base
A_C = np.diag(S_C) @ Vt_base

lora_matrix_A = B_A @ A_A
lora_matrix_B = B_B @ A_B
lora_matrix_C = B_C @ A_C

# Compute the ideal aggregated matrix
ideal_agg_matrix = w_A * lora_matrix_A + w_B * lora_matrix_B + w_C * lora_matrix_C

# --- Verification for Task 1 ---

# Compare the results of the two aggregation methods
is_equal = np.allclose(final_lora_agg_svd, ideal_agg_matrix, atol=1e-8)
print(f"Are the two aggregation results equivalent? {is_equal}")

Are the two aggregation results equivalent? True


In [12]:
# --- SVD Verification for Task 2 ---

# Use the result from Task 1 as the sample matrix
sample_matrix = final_lora_agg_svd

# Perform SVD decomposition
U_svd, S_svd, Vt_svd = np.linalg.svd(sample_matrix)

# Reconstruct the matrix from SVD components
S_matrix = np.zeros_like(sample_matrix, dtype=float)
S_matrix[:S_svd.shape[0], :S_svd.shape[0]] = np.diag(S_svd)
reconstructed_matrix = U_svd @ S_matrix @ Vt_svd

# Verify that the reconstruction is correct
is_reconstruction_correct = np.allclose(sample_matrix, reconstructed_matrix, atol=1e-8)
print(f"Is the SVD reconstruction correct? {is_reconstruction_correct}")

Is the SVD reconstruction correct? True
